In [1]:
import os
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
path_benchmark = "D:\\Projects\\Datasets\\Exams\\benchmark_final.avi"
path_result_excuse = "D:\\Projects\\Datasets\\Exams\\benchmark_final\\results\\result.txt"
path_result_deepRN = "D:\\Projects\\Results\\Qualificacao\\DeepRN\\label.csv"

In [3]:
file_excuse = pd.read_csv(path_result_excuse)
file_excuse

,img,position_x,position_y
0,1.png,238.000000,285.000000
1,10.png,222.000000,295.000000
2,100.png,191.000000,308.000000
3,1000.png,78.144545,419.346104
4,1001.png,280.000000,157.000000
...,...,...,...
4395,995.png,201.000000,280.000000
4396,996.png,0.000000,0.000000
4397,997.png,171.000000,299.000000
4398,998.png,287.100910,481.816606


In [4]:
file_deepRN = pd.read_csv(path_result_deepRN)
file_deepRN = file_deepRN.drop(['patient', 'param', 'radius', 'flash_algorithm', 'flash_information',
                                'color_flash', 'eye_size', 'img_mean', 'img_std', 'img_median'], axis=1)
file_deepRN

,frame,center_x,center_y
0,0,272.038625,222.280528
1,1,271.061452,223.986410
2,2,269.902751,226.316935
3,3,269.690399,228.427734
4,4,269.576441,229.833323
...,...,...,...
4395,4395,191.610158,162.026545
4396,4396,195.289307,162.813568
4397,4397,201.530584,158.175456
4398,4398,217.249247,143.932317


In [6]:
benchmark = cv2.VideoCapture(path_benchmark)

fourcc = cv2.VideoWriter_fourcc(*'MP42')
video = cv2.VideoWriter('compare_excuse_deepRN.avi', fourcc, float(17), (1280, 480))

ret, frame = benchmark.read()
number_frame = 0
while ret:
    filter_excuse = file_excuse['img'] == '{}.png'.format(number_frame + 1)
    data_excuse = file_excuse[filter_excuse]    
    center_excuse = (int(data_excuse['position_y'].values[0]), int(data_excuse['position_x'].values[0]))
    
    filter_deepRN = file_deepRN['frame'] == number_frame
    data_deepRN = file_deepRN[filter_deepRN]
    center_deepRN = (int(data_deepRN['center_x'].values[0]), int(data_deepRN['center_y'].values[0]))
    
    img_excuse = np.copy(frame)
    img_deepRN = np.copy(frame)
    
    if (center_excuse[0] > 0) and (center_excuse[1] > 0):
        cv2.line(img_excuse, (center_excuse[0], center_excuse[1] - 10), (center_excuse[0], center_excuse[1] + 10), (255, 255, 255), 2)
        cv2.line(img_excuse, (center_excuse[0] - 10, center_excuse[1]), (center_excuse[0] + 10, center_excuse[1]), (255, 255, 255), 2)
    
    if (center_deepRN[0] > 0) and (center_deepRN[1] > 0):
        cv2.line(img_deepRN, (center_deepRN[0], center_deepRN[1] - 10), (center_deepRN[0], center_deepRN[1] + 10), (255, 255, 255), 2)
        cv2.line(img_deepRN, (center_deepRN[0] - 10, center_deepRN[1]), (center_deepRN[0] + 10, center_deepRN[1]), (255, 255, 255), 2)

    cv2.putText(img_excuse, 'Excuse', (150, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(img_deepRN, 'DeepRN', (150, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    
    img_final = cv2.hconcat([img_excuse, img_deepRN])
    
    video.write(img_final)
    
    ret, frame = benchmark.read()
    number_frame += 1
    
video.release()